#deepfake from cruise

[아이언맨](https://www.youtube.com/watch?v=iDM69UEyM3w)

- Deep Learning으로 얼굴을 바꾸어 주는 툴로 소스코드는 막아 놓음
- 음성은 바꿀 수 있음 (인물의 5초 정도의 음성 녹음 파일이 필요)
- A 영상의 인물을 B 인물의 사진 한장으로 바꿔 치기 (눈, 코, 입 주변만)




[Ai 앵커](https://www.youtube.com/watch?v=5GFt5TAmPfk)

# Demo for paper "First Order Motion Model for Image Animation"


**Clone repository**

In [1]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 299, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 299 (delta 2), reused 4 (delta 1), pack-reused 293
Receiving objects: 100% (299/299), 72.15 MiB | 15.72 MiB/s, done.
Resolving deltas: 100% (153/153), done.


In [2]:
cd first-order-model

/content/first-order-model


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Load driving video and source image**

In [6]:

import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import skimage
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

source_image = imageio.imread('/content/drive/MyDrive/myface.JPG')
#source_image = imageio.imread('02.png')
reader = imageio.get_reader('/content/04.mp4')
#reader = imageio.get_reader('04.mp4')


#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]

fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()

driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

**Create a model and load checkpoints**

In [8]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/drive/My Drive/vox-cpk.pth.tar')

**Perform image animation**

In [9]:
from demo import make_animation
from skimage import img_as_ubyte


predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 211/211 [00:08<00:00, 24.17it/s]
